# py0xcluster

This is for now an educational project for myself with the aim of performing fun data-science projects around blockchain data gathered through the Graph Network (https://thegraph.com/)

## Target objectives

- Establish meaningful grouping of address by clustering DEX traders and LP
    - Feature Extraction:
        - TBD but based on mint/swap/burn data from messari subgraphs entities
        - with or without balances at swap time (web3py fetch balance at block)
        - EOA vs Contracts
    - Dimensionality reduction:
        - UMAP / tSNE or PCA / ICA
    - Clustering:
        - DBSCAN
        - silhouette evaluation
    - Visualization:
        - scatter plot with color-coded returns? (TBD)

**Secondary objective: identify which group has the most profitable activity**

- Triggered Average of price by swap in/out by group of addresses

- Predict future returns based on the activity of previously clustered groups of addresses

## Random list of potential features:

### Addresses
- z-scored (clarify how) difference of price 24h? after swap -> could be target independant variable
- % of Limit order on uni-v3 (one deposit amout = 0)
- is contract
- nb of events (z-scored to other addresses on same pool)
- average swap size (z-scored/pool)
- average deposit size (z-scored/pool)
- average withdraw size (z-score/pool)
- ratio? of nb of: swaps / (deposits + withdraws)
- nb of liquid pools interacted with
- % of events (likely swaps) happening in the same block (possibly identical to MEV bots?)




### Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from py0xcluster.utils.query_utils import *
from py0xcluster.main_classes.pools import *
from py0xcluster.main_classes.pool_events import *

### Gathering data about most-active pools
    - need to adapt to take into account refactoring of queries-related methods for multiple entities at once

In [4]:
uni3pools_selector = PoolSelector(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    min_daily_volume_USD = 100000,
    min_TVL = 100000, # Not implemented. consider removing
    start_date = (2022,12,21), 
    end_date = (2023,1,10),
    days_batch_size = 20)

uni3_pools = uni3pools_selector.create_pool_selection(verbose=True)
uni3_pools.pools_df

Queriying from 2022-12-21 00:00:00 to 2023-01-10 00:00:00
2650 lquidity pools snapshots retrieved
357 stable pools snapshots (over 2650) have been removed
274 illiquid pools snapshots (over 2650) have been removed 
221 pools were selected


,pool.name,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD,pool.protocol.name,pool.protocol.network,pool.id,token0.symbol,token1.symbol
0,Uniswap V3 USD Coin/Wrapped Ether 0.05%,1.407573e+08,1.844735e+08,1.000000,1419.860391,Uniswap V3,MAINNET,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH
1,Uniswap V3 USD Coin/Tether USD 0.01%,7.632963e+07,1.199310e+08,1.000000,1.000000,Uniswap V3,MAINNET,0x3416cf6c708da44db2624d63ea0aaef7113527c6,USDC,USDT
2,Uniswap V3 Wrapped Ether/Tether USD 0.05%,2.748191e+07,1.812181e+07,1419.860391,1.000000,Uniswap V3,MAINNET,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,USDT
3,Uniswap V3 USD Coin/Wrapped Ether 0.01%,1.951618e+07,2.979251e+06,1.000000,1419.860391,Uniswap V3,MAINNET,0xe0554a476a092703abdb3ef35c80e0d76d32939f,USDC,WETH
4,Uniswap V3 Wrapped BTC/Wrapped Ether 0.05%,1.490067e+07,1.196053e+08,18804.772271,1419.860391,Uniswap V3,MAINNET,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,WETH
...,...,...,...,...,...,...,...,...,...,...
216,Uniswap V3 Green/Wrapped Ether 0.05%,1.030147e+05,6.334998e+05,0.002009,1419.860391,Uniswap V3,MAINNET,0x4efc9e2e3e77732ce2f9612b8f050082c01688bd,GREEN,WETH
217,Uniswap V3 HuntToken/Tether USD 0.3%,1.028683e+05,1.019830e+06,0.252896,1.000000,Uniswap V3,MAINNET,0x54578b6f942aeb23b67a8cef24220651306b8e26,HUNT,USDT
218,Uniswap V3 Index/Wrapped Ether 1%,1.026767e+05,2.350162e+04,0.000000,1419.860391,Uniswap V3,MAINNET,0x8c13148228765ba9e84eaf940b0416a5e349a5e7,INDEX,WETH
219,Uniswap V3 unification.com/xfund/Wrapped Ether 1%,1.023294e+05,1.365642e+04,0.000000,1419.860391,Uniswap V3,MAINNET,0xb1223da8a5929bcfa9d26f0c6da8f0a29c3925ff,xFUND,WETH


## Extracting all events from these pools

### Steps

- Data query:
    - Perform query for each pool, batch by days to accomodate response limit
    - Alternatively the query can be done on multiple pools. Ideally, the size of the batch should be proportional to the volume, but quite arbitrary to implement.
    - Consider whether do swaps / mints / burns separately or jointly
    - Loop and aggregate over days / pools (or batch of pools)


In [78]:
uni3_events_getter = PoolEventGetter(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    pool_ids = uni3_pools.pools_df['pool.id'][47:50],
    start_date = (2023,1,1), 
    end_date = (2023,1,11),
    days_batch_size = 5
    )

# Get swaps, deposits, and withdraw from a pool.
uni3_events = uni3_events_getter.get_events(verbose=True)


pool: 0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
Queriying from 2023-01-01 00:00:00 to 2023-01-06 00:00:00
Queriying from 2023-01-06 00:00:00 to 2023-01-11 00:00:00
pool: 0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
Queriying from 2023-01-01 00:00:00 to 2023-01-06 00:00:00
Queriying from 2023-01-06 00:00:00 to 2023-01-11 00:00:00
pool: 0xe68acf3a42b74b0d7609330661732b08a28febee
Queriying from 2023-01-01 00:00:00 to 2023-01-06 00:00:00
Queriying from 2023-01-06 00:00:00 to 2023-01-11 00:00:00


In [79]:
uni3_events['deposits']

,amountUSD,from,to,timestamp,blockNumber,InputTokenAmount0,InputTokenAmount1,pool.id
0,5.702699447989583112621271946559613,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597,1672546583,16309467,3999999999999999546796336914,4775822800551261,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
1,5.702699447989583112621271946559613,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597,1672546583,16309467,3999999999999999546796336914,4775822800551261,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
2,914851.765461960850048108842731207,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1672528319,16307950,2218065428,549646359583,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
3,7631.111087067667674748898327938856,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1672807583,16331122,24040490,3579985456,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
4,1479.361575289250415460937779284362,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1672678103,16320383,5095577,627883205,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
...,...,...,...,...,...,...,...,...
63,47.34801450889457948824739015257494,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673285687,16370771,133568,24169363,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
64,60048.59240125194199882926006636583,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673384231,16378942,171548339,30157522440,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
65,30777.51020403673075550783963036063,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673257451,16368425,24722267,26499999181,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
66,19448295.85831377786226568728901846,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673279639,16370267,279198452,19399767114407,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35


In [77]:
uni3_events['deposits']

,amountUSD,from,to,timestamp,blockNumber,0,1,pool.id
0,5.702699447989583112621271946559613,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597,1672546583,16309467,3999999999999999546796336914,4775822800551261,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
1,5.702699447989583112621271946559613,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597,1672546583,16309467,3999999999999999546796336914,4775822800551261,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
2,914851.765461960850048108842731207,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1672528319,16307950,2218065428,549646359583,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
3,7631.111087067667674748898327938856,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1672807583,16331122,24040490,3579985456,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
4,1479.361575289250415460937779284362,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1672678103,16320383,5095577,627883205,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
...,...,...,...,...,...,...,...,...
63,47.34801450889457948824739015257494,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673285687,16370771,133568,24169363,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
64,60048.59240125194199882926006636583,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673384231,16378942,171548339,30157522440,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
65,30777.51020403673075550783963036063,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673257451,16368425,24722267,26499999181,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
66,19448295.85831377786226568728901846,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,1673279639,16370267,279198452,19399767114407,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35


In [75]:
uni3_events['withdraws']

,amountUSD,from,to,timestamp,blockNumber,0,1,pool.id
0,8.632159831077257412017026900253474,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597,0xc36442b4a4522e871399cd717abdd847ab11fe88,1672650635,16318105,NaN,NaN,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
1,16.46073019984436402010455250732899,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597,0xc36442b4a4522e871399cd717abdd847ab11fe88,1672650599,16318102,NaN,NaN,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
2,8.632159831077257412017026900253474,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597,0xc36442b4a4522e871399cd717abdd847ab11fe88,1672650635,16318105,55818799545731129819759935,7087183718750305,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
3,16.46073019984436402010455250732899,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597,0xc36442b4a4522e871399cd717abdd847ab11fe88,1672650599,16318102,40993619153442777383295809,13514603685983850,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
4,NaN,NaN,NaN,NaN,NaN,55818799545731129819759935,7087183718750305,0xb82d251b7e92832dccb7f3c6f03ae20f2d7ee597
...,...,...,...,...,...,...,...,...
107,31.24717541235066427325976126397915,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,0xbd7c6d2ede836b6b27c461799c4e9ecb8f4e8a66,1673256263,16368327,28507,26328938,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
108,25.07851205114872711984067970485044,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,0x79511dadf55edb9d0ed490e3fcd1ea2cbea62203,1673129675,16357822,73022,12755078,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
109,8952.999577653738038941966518582192,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,0x7902d8e4b6e9ad7e4ecdc5117b6b20ce01d3cfb7,1673334839,16374841,105057,8934887233,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
110,NaN,NaN,NaN,NaN,NaN,528844875,94784033938,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35


In [ ]:
data_lengths = [0, 0, 0]
empty_data = [data_length == 0 for data_length in data_lengths]
all(empty_data)

True

In [ ]:
dico = {'ac': 0 , 'asfd': 2}
len(dico)

2